In [12]:
import pandas as pd
from predict_winner import read_in_all_matches

def declare_winner(df: pd.DataFrame) -> pd.DataFrame:
    df['map_winner'] = ["host" if a > b else "guest" for a, b in zip(df['matchGameResult.hostGameScore'], df['matchGameResult.guestGameScore'])]

    df['is_winner'] = ["Y" if a == b else "N" for a, b in zip(df['map_winner'], df['team_type'])]

    return df

def head_to_head(data, main, opposition):
    data = declare_winner(data)
    grouping = {a: 'sum' for a in ['totalDamageDealt', 'totalKills', 'totalDeaths', 'totalFirstBloodKills', 'totalRotationKills', 'totalShotsHit', 'totalShotsFired']}
    grouping2 = {b: 'first' for b in ['team_type', 'matchGameResult.guestGameScore', 'matchGameResult.hostGameScore']}
    full_group = {**grouping , **grouping2}
    group = data.groupby(['gameMap', 'oppo_abbrev', 'abbrev', 'gameMode', 'map_winner', 'matchGame.matchId']).agg(full_group)

    group['rounds'] = group['matchGameResult.guestGameScore']+group['matchGameResult.hostGameScore']
    group['teamKd'] = group['totalKills']/group['totalDeaths']
    group['accuracy'] = group['totalShotsHit']/group['totalShotsFired']
    group['rotationalPercent'] = group['totalRotationKills']/group['totalKills']
    group['fbPerc'] = group['totalFirstBloodKills']/group['rounds']

    group.reset_index(inplace=True)
    group.drop(columns=['totalShotsHit', 'totalShotsFired', 'totalKills', 'totalDeaths', 'totalRotationKills', 'matchGameResult.hostGameScore', 
                        'matchGameResult.guestGameScore', 'totalFirstBloodKills'], inplace=True)
    
    data_opp = group[group['oppo_abbrev']==opposition]
    data_opp = data_opp[data_opp['abbrev']== main]
    data_opp2 = group[group['abbrev']==opposition]
    data_opp2 = data_opp2[data_opp2['oppo_abbrev']== main].drop(columns=['oppo_abbrev', 'abbrev', 'team_type'])

    joined = pd.merge(data_opp, data_opp2, on=['matchGame.matchId', 'gameMap', 'gameMode'], suffixes=('_team1', '_team2'))

    snd_a = joined[joined['gameMode']=='CDL SnD'].drop(columns=['rotationalPercent_team1', 'rotationalPercent_team2'],
                                                       axis=1).dropna(how='any', axis=0).groupby('gameMode').mean().drop('matchGame.matchId', axis=1)
    hp_a = joined[joined['gameMode']=='CDL SnD'].drop(columns=['fbPerc_team1', 'fbPerc_team2'],
                                                      axis=1).dropna(how='any', axis=0).groupby('gameMode').mean().drop('matchGame.matchId', axis=1)
    control_a = joined[joined['gameMode']=='CDL Control'].drop(columns=['rotationalPercent_team1', 'rotationalPercent_team2', 'fbPerc_team1',
                                                                        'fbPerc_team2'], axis=1).dropna(how='any', axis=0).groupby('gameMode').mean().drop('matchGame.matchId', axis=1)

    data = read_in_all_matches()
    data = declare_winner(data)
    
    home = data[data['abbrev']==main]
    home = home[home['oppo_abbrev']!=opposition]
    grouping = {a: 'sum' for a in ['totalDamageDealt', 'totalKills', 'totalDeaths', 'totalFirstBloodKills', 'totalRotationKills', 'totalShotsHit', 'totalShotsFired']}
    grouping2 = {b: 'first' for b in ['team_type', 'matchGameResult.guestGameScore', 'matchGameResult.hostGameScore', 'matchGame.updatedAt']}
    full_group = {**grouping , **grouping2}
    group = home.groupby(['gameMap', 'oppo_abbrev', 'abbrev', 'gameMode', 'map_winner', 'matchGame.matchId']).agg(full_group)

    group['rounds'] = group['matchGameResult.guestGameScore']+group['matchGameResult.hostGameScore']
    group['teamKd'] = group['totalKills']/group['totalDeaths']
    group['accuracy'] = group['totalShotsHit']/group['totalShotsFired']
    group['rotationalPercent'] = group['totalRotationKills']/group['totalKills']
    group['fbPerc'] = group['totalFirstBloodKills']/group['rounds']

    group.reset_index(inplace=True)
    group.drop(columns=['totalShotsHit', 'totalShotsFired', 'totalKills', 'totalDeaths', 'totalRotationKills', 'matchGameResult.hostGameScore', 
                        'matchGameResult.guestGameScore', 'totalFirstBloodKills', 'rounds', 'team_type', 'map_winner'], inplace=True)
    group = group.sort_values('matchGame.updatedAt', ascending=False)
    group = group[0:15]

    snd = group[group['gameMode']=='CDL SnD'].drop(columns=['rotationalPercent'], 
                                                   axis=1).dropna(how='any', axis=0).groupby('gameMode').mean().drop(columns=['matchGame.matchId', 'matchGame.updatedAt'],
                                                                                                                         axis=1)
    hp = group[group['gameMode']=='CDL SnD'].drop(columns=['fbPerc'],
                                                   axis=1).dropna(how='any', axis=0).groupby('gameMode').mean().drop(columns=['matchGame.matchId', 'matchGame.updatedAt'],
                                                                                                                         axis=1)
    control = group[group['gameMode']=='CDL Control'].drop(columns=['rotationalPercent',
                                                                     'fbPerc'], axis=1).dropna(how='any', axis=0).groupby('gameMode').mean().drop(columns=['matchGame.matchId', 'matchGame.updatedAt'],
                                                                                                                         axis=1)

    home = data[data['abbrev']==opposition]
    home = home[home['oppo_abbrev']!=main]
    grouping = {a: 'sum' for a in ['totalDamageDealt', 'totalKills', 'totalDeaths', 'totalFirstBloodKills', 'totalRotationKills', 'totalShotsHit', 'totalShotsFired']}
    grouping2 = {b: 'first' for b in ['team_type', 'matchGameResult.guestGameScore', 'matchGameResult.hostGameScore', 'matchGame.updatedAt']}
    full_group = {**grouping , **grouping2}
    group = home.groupby(['gameMap', 'oppo_abbrev', 'abbrev', 'gameMode', 'map_winner', 'matchGame.matchId']).agg(full_group)

    group['rounds'] = group['matchGameResult.guestGameScore']+group['matchGameResult.hostGameScore']
    group['teamKd'] = group['totalKills']/group['totalDeaths']
    group['accuracy'] = group['totalShotsHit']/group['totalShotsFired']
    group['rotationalPercent'] = group['totalRotationKills']/group['totalKills']
    group['fbPerc'] = group['totalFirstBloodKills']/group['rounds']

    group.reset_index(inplace=True)
    group.drop(columns=['totalShotsHit', 'totalShotsFired', 'totalKills', 'totalDeaths', 'totalRotationKills', 'matchGameResult.hostGameScore', 
                        'matchGameResult.guestGameScore', 'totalFirstBloodKills', 'rounds', 'team_type', 'map_winner'], inplace=True)
    group = group.sort_values('matchGame.updatedAt', ascending=False)
    group = group[0:20]

    snd_op = group[group['gameMode']=='CDL SnD'].drop(columns=['rotationalPercent'],
                                                       axis=1).dropna(how='any', axis=0).groupby('gameMode').mean().drop(columns=['matchGame.matchId', 'matchGame.updatedAt'],
                                                                                                                         axis=1)
    hp_op = group[group['gameMode']=='CDL SnD'].drop(columns=['fbPerc'],
                                                      axis=1).dropna(how='any', axis=0).groupby('gameMode').mean().drop(columns=['matchGame.matchId', 'matchGame.updatedAt'],
                                                                                                                         axis=1)
    control_op = group[group['gameMode']=='CDL Control'].drop(columns=['rotationalPercent',
                                                                        'fbPerc'], axis=1).dropna(how='any', axis=0).groupby('gameMode').mean().drop(columns=['matchGame.matchId', 'matchGame.updatedAt'],
                                                                                                                         axis=1)

    snd_j = pd.merge(snd, snd_op, on='gameMode', suffixes=('_team1', '_team2'))
    hp_j = pd.merge(hp, hp_op, on='gameMode', suffixes=('_team1', '_team2'))
    control_j = pd.merge(control, control_op, on='gameMode', suffixes=('_team1', '_team2'))

    search = pd.concat([snd_a, snd_j]).drop(columns=['rounds_team1', 'rounds_team2'], axis=1).groupby('gameMode').mean()
    hard = pd.concat([hp_a, hp_j]).drop(columns=['rounds_team1', 'rounds_team2'], axis=1).groupby('gameMode').mean()
    cntl = pd.concat([control_a, control_j]).drop(columns=['rounds_team1', 'rounds_team2'], axis=1).groupby('gameMode').mean()


    return search, hard, cntl


d = read_in_all_matches()
s, h, c = head_to_head(d, "MIN", "NY")

In [16]:
from predict_winner import hardpoint_model, control_model, snd_model


hardpoint_model(h)

array([1])

In [14]:
control_model(c)

array([1])

In [15]:
snd_model(s)

array([1])